<a href="https://colab.research.google.com/github/tumble-weed/LSTM-from-scratch/blob/master/GRU_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch

In [0]:
seq_len,batch_size,feat_len = 5,32,100
seq = np.random.random((seq_len,batch_size,feat_len))
seq = torch.tensor(seq).float()

In [0]:
class GRU(torch.nn.Module):
    def __init__(self,input_len,output_len):
        
        torch.nn.Module.__init__(self)
        
        hidden_len = output_len
        
        self.hidden_len = hidden_len
        self.output_len = output_len
        self.input_len = input_len
        #---------------------------------------------------------
        # Gates
        #---------------------------------------------------------
        self.write_gate = torch.nn.Linear(hidden_len,hidden_len)
        self.read_gate = torch.nn.Linear(hidden_len,hidden_len)
        #---------------------------------------------------------
        # fc
        #---------------------------------------------------------
        self.fc_input = torch.nn.Linear(input_len,output_len)
        self.fc_state = torch.nn.Linear(hidden_len,output_len,bias = False)
    
    def forward(self,x,state):
        outs = []
        for at_t in x:
            read = torch.nn.functional.sigmoid(self.read_gate(state))
            write = torch.nn.functional.sigmoid(self.write_gate(state))
            
            pre_state = self.fc_state(state*read) + self.fc_input(at_t)
            state = write*pre_state + (1-write)*state
            out = state
            outs.append(out)
        outs = torch.stack(outs,0)
        return outs,state